# Developing an Image Classifier Using Synthetic Data from CAD Models
### Part I of Berkeley PCMLAI - Final Capstone Project
#### A.Morato

In the first part of the project, a dataset of artificial images is created. The process starts with 5 CAD models (.stp files), one for each class of magnet we want to classify. The tools used to process CAD files come from free platform called FreeCAD and PyVista, which work exclusively with .stl models. An .stl file essentially contains only the surface information of the object. The surface models are then scanned using a moving plotter provided by PyVista, which offers a high-level API for 3D visualization and data analysis.

The following 5 different magnets are processed:

- QD - Defocusing quadrupole magnet (Berkeley Lab code AL-1115-5731)
- QF - Focusing quadrupole magnet (AL-1115-6143)
- QFA - Quadrupole focusing alternating magnet (AL-1114-0240)
- SD - Sextupole defocusing magnet (AL-1119-9432)
- SHD - Sextupole horizontal defocusing magnet (AL-1154-3320)

In [1]:
!pip install pyvista

In [14]:
import os
import sys
import glob
import pyvista as pv

In [4]:
# Adding FreeCAD bin and lib directories to the system path, to ensure that Python can access FreeCAD's executables and libraries

# Path to the FreeCAD bin directory
freecad_bin_path = r"C:\Program Files\FreeCAD 0.21\bin"

# Add FreeCAD bin path to system path
if freecad_bin_path not in sys.path:
    sys.path.append(freecad_bin_path)

# Path to the FreeCAD lib directory (if required)
freecad_lib_path = r"C:\Program Files\FreeCAD 0.21\lib"

# Add FreeCAD lib path to system path
if freecad_lib_path not in sys.path:
    sys.path.append(freecad_lib_path)

In [5]:
# FreeCAD can now be imported
import FreeCAD

### 1. Generation of .stl files
The first step is collecting the CAD models of the selected magnets (.stp files) and convert them to a format suitable for FreeCAD (.stl)

In [6]:
import FreeCAD as App
import Part
import Mesh
from typing import List

In [11]:
# Definition of a "Converter" class to handle converting STEP files to STL files, including methods for file handling and conversion.
# The main function handles argument parsing, initializes the Converter, and starts the conversion process.

class Converter(object):
    """A class that can convert STEP files to STL files.

    Attributes:
        file_list (List[str]): A list of file names to convert.
        path (str): The path to the folder containing the input files.

    Methods:
        get_all_files_in_folder(path: str = './') -> List[str]:
            Returns a list of all file names in the specified folder.

        convert(output_path: str = None):
            Converts each file in the 'file_list' from STEP format to STL format.

    Usage:
        # Example usage of the Converter class
        file_list = ['file1.step', 'file2.STEP', 'file3.stp']
        converter = Converter(file_list, path='./input_folder/')
        converter.convert(output_path='./output_folder/')
    """

    def __init__(self, file_list: List[str], path: str = './'):
        """Initialize the Converter object.

        Args:
            file_list (List[str]): A list of file names to convert.
            path (str, optional): The path to the folder containing the input files.
                Defaults to './'.
        """
        super().__init__()
        self.file_list = file_list
        self.path = path

    @classmethod
    def get_all_files_in_folder(cls, path: str = './') -> List[str]:
        """Get a list of all file names in the specified folder.

        Args:
            path (str, optional): The path to the folder. Defaults to './'.

        Returns:
            List[str]: A list of all file names in the specified folder.
        """
        return [os.path.basename(path) for path in glob.glob(os.path.join(path, '*.*'))]

    @classmethod
    def format_time_elapsed(cls, start_time: datetime, end_time: datetime) -> str:
        """Format the elapsed time between two datetime objects into a human-readable string.

        Args:
            start_time (datetime): The starting time.
            end_time (datetime): The ending time.

        Returns:
            str: A human-readable string representing the elapsed time.
        """
        elapsed_time = end_time - start_time
        hours, remainder = divmod(elapsed_time.total_seconds(), 3600)
        minutes, seconds = divmod(remainder, 60)

        formatted_time = ""
        if hours > 0:
            formatted_time += f"{int(hours)} hours, "
        if minutes > 0:
            formatted_time += f"{int(minutes)} minutes, "
        formatted_time += f"{int(seconds)} seconds"

        return formatted_time

    def convert(self, output_path: str = None):
        """Convert the files from STEP format to STL format.

        Args:
            output_path (str, optional): The path to the folder where the output files will be saved.
                If not provided, the output files will be saved in the same folder as the input files.
        """
        start_time = datetime.now()
        for file in self.file_list:
            print('\033[34m' + 'Processing file: ' + file + '\033[0m')
            shape = Part.Shape()
            shape.read(os.path.join(self.path, file))
            doc = App.newDocument('Doc')
            pf = doc.addObject("Part::Feature", "MyShape")
            pf.Shape = shape
            output_filename = file
            for suffix in ['.step', '.stp', '.STEP', '.STP']:
                output_filename = output_filename.replace(suffix, '.stl')
            if output_path is None:
                Mesh.export([pf], os.path.join(self.path, output_filename))
            else:
                Mesh.export([pf], os.path.join(output_path, output_filename))
        end_time = datetime.now()
        print('\033[32m' + '\nDone!' + '\033[0m')
        print('\033[32m' + 'Elapsed time: ' + Converter.format_time_elapsed(start_time, end_time) + '\033[0m')

In [24]:
# List of .stp files to load
filelist=['al-1115-5731_AQD.stp', 'al-1115-6143_AQF.stp', 'al-1114-0240_AQFA.stp', 'al-1119-9432_ASD.stp', 'al-1154-3320_ASHD.stp']

In [25]:
# Convertion of .stp files to .stl files
input_folder = 'data/FILES_STP'
output_folder = 'data/FILES_STL'

# Create the Converter object and convert the files
converter = Converter(filelist, path=input_folder)
print('\033[32m' + 'Starting conversion loop...\n' + '\033[0m')
converter.convert(output_path=output_folder)

Starting conversion loop...

Processing file: al-1115-5731_AQD.stp
Processing file: al-1115-6143_AQF.stp
Processing file: al-1114-0240_AQFA.stp
Processing file: al-1119-9432_ASD.stp
Processing file: al-1154-3320_ASHD.stp

Done!
Elapsed time: 12 minutes, 55 seconds


### 2. Creation of a Collection of .png Pictures
Each .stl file is loaded into PyVista to generate a mesh. A plotter is then created to navigate around the generated mesh and capture images from different angles. The rotation primarily involves moving around the object, with approximately ±40 degrees in height, and varying angles to tilt the plotter, emulating a slightly misaligned image. Parameters involving advanced features (e.g., lighting, surface material) are present but have not been considered at this stage. I also chose to process each magnet separately rather than using a common loop. The main reason for this is that the code was run on my personal computer, and this approach allowed me to split the time-consuming process into different sessions.

#### 2.1 AL-1115-5731 - QD

In [26]:
# Define work folders
INPUT_FILE = 'data/STL/al-1115-5731_AQD.stl'
OUTPUT_DIR = 'data/CAD_pics'

# Load the model
print(f'Reading {INPUT_FILE}...')
mesh = pv.read(INPUT_FILE)
print(f'DONE! Starting loop...')

Reading data/STL/al-1115-5731_AQD.stl...
DONE! Starting loop...


In [27]:
#Check the object view
plotter = pv.Plotter(off_screen=True)
plotter.background_color = 'white'  # Set a white background color

# Render the scene
plotter.add_mesh(mesh, show_edges=False, specular=1)  # Set color and specular properties
# Set the viewing angle in another plane
plotter.camera_position = 'xy'
plotter.camera.azimuth = 120
    
#plotter.show(auto_close=False)
plotter.screenshot('test.png')
    
# Close the plotter
plotter.close()

In [29]:
# Loop through different viewing angles and save the rendered images
for azim_angle in range(0, 360, 15):
    for elev_angle in range(-30, 50, 10):  
        
        #'Roll' to emulate not ideal alligned pictures
        for roll_angle in range(-20, 30, 10):
            
            # Set up a plotter for rendering
            plotter = pv.Plotter(off_screen=True)
            plotter.background_color = 'white'  # Set a white background color

            # Render the scene
            #plotter.add_mesh(mesh, show_edges=False, color='gray', specular=1.0)  # Set color and specular properties

            # Check if the mesh has color data
            if 'colors' in mesh.point_data:
                # Use the existing colors
                plotter.add_mesh(mesh, show_edges=False, specular=1)
            else:
                # Apply a default color if no colors data
                plotter.add_mesh(mesh, color='gray', show_edges=False, specular=1)

            # Set the viewing angle
            plotter.camera_position = 'xy'
            plotter.camera.azimuth += azim_angle
            plotter.camera.elevation += elev_angle
            plotter.camera.roll += roll_angle
            #print(angle)

            #DISABLED OUTPUT PREVIEW
            #plotter.show(auto_close=False)

            # Save the rendered image
            filename = f'QD_rot_a{azim_angle}_e{elev_angle}_r{roll_angle}.png'
            output_file = os.path.join(OUTPUT_DIR, filename)
            plotter.screenshot(output_file)
                
            # Close the plotter
            plotter.close()

#### 2.2 AL-1115-6143 - QF

In [ ]:
# Define work folders
INPUT_FILE = 'data/STL/al-1115-6143_AQF.stl'
OUTPUT_DIR = 'data/CAD_pics'

# Load the model
print(f'Reading {INPUT_FILE}...')
mesh = pv.read(INPUT_FILE)
print(f'DONE! Starting loop...')

Reading data/STL/al-1115-6143_AQF.stl...
DONE! Starting loop...


In [ ]:
#Check the object view
plotter = pv.Plotter(off_screen=True)
plotter.background_color = 'white'  # Set a white background color

# Render the scene
plotter.add_mesh(mesh, show_edges=False, specular=1)  # Set color and specular properties
# Set the viewing angle in another plane
plotter.camera_position = 'xy'
plotter.camera.azimuth = 120
    
#plotter.show(auto_close=False)
plotter.screenshot('test.png')
    
# Close the plotter
plotter.close()

In [ ]:
# Loop through different viewing angles and save the rendered images
for azim_angle in range(0, 360, 15):
    for elev_angle in range(-30, 50, 10):  
        
        #'Roll' to emulate not ideal alligned pictures
        for roll_angle in range(-20, 30, 10):
            
            # Set up a plotter for rendering
            plotter = pv.Plotter(off_screen=True)
            plotter.background_color = 'white'  # Set a white background color

            # Render the scene
            #plotter.add_mesh(mesh, show_edges=False, color='gray', specular=1.0)  # Set color and specular properties

            # Check if the mesh has color data
            if 'colors' in mesh.point_data:
                # Use the existing colors
                plotter.add_mesh(mesh, show_edges=False, specular=1)
            else:
                # Apply a default color if no colors data
                plotter.add_mesh(mesh, color='gray', show_edges=False, specular=1)

            # Set the viewing angle
            plotter.camera_position = 'xy'
            plotter.camera.azimuth += azim_angle
            plotter.camera.elevation += elev_angle
            plotter.camera.roll += roll_angle
            #print(angle)

            #DISABLED OUTPUT PREVIEW
            #plotter.show(auto_close=False)

            # Save the rendered image
            filename = f'QF_rot_a{azim_angle}_e{elev_angle}_r{roll_angle}.png'
            output_file = os.path.join(OUTPUT_DIR, filename)
            plotter.screenshot(output_file)
                
            # Close the plotter
            plotter.close()

#### 2.3 AL-1114-0240 - QFA


In [6]:
# Define work folders
INPUT_FILE = 'data/STL/al-1114-0240_AQFA.stl'
OUTPUT_DIR = 'data/CAD_pics'

# Load the model
print(f'Reading {INPUT_FILE}...')
mesh = pv.read(INPUT_FILE)
print(f'DONE! Starting loop...')

Reading data/STL/al-1114-0240_AQFA.stl...
DONE! Starting loop...


In [7]:
#Check the object view
plotter = pv.Plotter(off_screen=True)
plotter.background_color = 'white'  # Set a white background color

# Render the scene
plotter.add_mesh(mesh, show_edges=False, specular=1)  # Set color and specular properties
# Set the viewing angle in another plane
plotter.camera_position = 'xy'
plotter.camera.azimuth = 120
    
#plotter.show(auto_close=False)
plotter.screenshot('test.png')
    
# Close the plotter
plotter.close()

In [15]:
# Loop through different viewing angles and save the rendered images
for azim_angle in range(0, 360, 15):
    for elev_angle in range(-30, 50, 10):  
        
        #'Roll' to emulate not ideal alligned pictures
        for roll_angle in range(-20, 30, 10):
            
            # Set up a plotter for rendering
            plotter = pv.Plotter(off_screen=True)
            plotter.background_color = 'white'  # Set a white background color

            # Render the scene
            #plotter.add_mesh(mesh, show_edges=False, color='gray', specular=1.0)  # Set color and specular properties

            # Check if the mesh has color data
            if 'colors' in mesh.point_data:
                # Use the existing colors
                plotter.add_mesh(mesh, show_edges=False, specular=1)
            else:
                # Apply a default color if no colors data
                plotter.add_mesh(mesh, color='gray', show_edges=False, specular=1)

            # Set the viewing angle
            plotter.camera_position = 'xy'
            plotter.camera.azimuth += azim_angle
            plotter.camera.elevation += elev_angle
            plotter.camera.roll += roll_angle
            #print(angle)

            #DISABLED OUTPUT PREVIEW
            #plotter.show(auto_close=False)

            # Save the rendered image
            filename = f'QFA_rot_a{azim_angle}_e{elev_angle}_r{roll_angle}.png'
            output_file = os.path.join(OUTPUT_DIR, filename)
            plotter.screenshot(output_file)
                
            # Close the plotter
            plotter.close()

#### 2.4 AL-1119-9432 - SD

In [22]:
# Define work folders
INPUT_FILE = 'data/STL/al-1119-9432_ASD.stl'
OUTPUT_DIR = 'data/CAD_pics'

# Load the model
print(f'Reading {INPUT_FILE}...')
mesh = pv.read(INPUT_FILE)
print(f'DONE! Starting loop...')

Reading data/STL/al-1119-9432_ASD.stl...
DONE! Starting loop...


In [23]:
#Check the object view
plotter = pv.Plotter(off_screen=True)
plotter.background_color = 'white'  # Set a white background color

# Render the scene
plotter.add_mesh(mesh, show_edges=False, specular=1)  # Set color and specular properties
# Set the viewing angle in another plane
plotter.camera_position = 'xy'
plotter.camera.azimuth = 120
    
#plotter.show(auto_close=False)
plotter.screenshot('test.png')
    
# Close the plotter
plotter.close()

In [24]:
# Loop through different viewing angles and save the rendered images
for azim_angle in range(0, 360, 15):
    for elev_angle in range(-30, 50, 10):  
        
        #'Roll' to emulate not ideal alligned pictures
        for roll_angle in range(-20, 30, 10):
            
            # Set up a plotter for rendering
            plotter = pv.Plotter(off_screen=True)
            plotter.background_color = 'white'  # Set a white background color

            # Render the scene
            #plotter.add_mesh(mesh, show_edges=False, color='gray', specular=1.0)  # Set color and specular properties

            # Check if the mesh has color data
            if 'colors' in mesh.point_data:
                # Use the existing colors
                plotter.add_mesh(mesh, show_edges=False, specular=1)
            else:
                # Apply a default color if no colors data
                plotter.add_mesh(mesh, color='gray', show_edges=False, specular=1)

            # Set the viewing angle
            plotter.camera_position = 'xy'
            plotter.camera.azimuth += azim_angle
            plotter.camera.elevation += elev_angle
            plotter.camera.roll += roll_angle
            #print(angle)

            #DISABLED OUTPUT PREVIEW
            #plotter.show(auto_close=False)

            # Save the rendered image
            filename = f'SD_rot_a{azim_angle}_e{elev_angle}_r{roll_angle}.png'
            output_file = os.path.join(OUTPUT_DIR, filename)
            plotter.screenshot(output_file)
                
            # Close the plotter
            plotter.close()

#### 2.5 AL-1154-3320 - SHD

In [25]:
# Define work folders
INPUT_FILE = 'data/STL/al-1154-3320_ASHD.stl'
OUTPUT_DIR = 'data/CAD_pics'

# Load the model
print(f'Reading {INPUT_FILE}...')
mesh = pv.read(INPUT_FILE)
print(f'DONE! Starting loop...')

Reading data/STL/al-1154-3320_ASHD.stl...
DONE! Starting loop...


In [26]:
#Check the object view
plotter = pv.Plotter(off_screen=True)
plotter.background_color = 'white'  # Set a white background color

# Render the scene
plotter.add_mesh(mesh, show_edges=False, specular=1)  # Set color and specular properties
# Set the viewing angle in another plane
plotter.camera_position = 'xy'
plotter.camera.azimuth = 120
    
#plotter.show(auto_close=False)
plotter.screenshot('test.png')
    
# Close the plotter
plotter.close()

In [28]:
# Loop through different viewing angles and save the rendered images
for azim_angle in range(0, 360, 15):
    for elev_angle in range(-30, 50, 10):  
        
        #'Roll' to emulate not ideal alligned pictures
        for roll_angle in range(-20, 30, 10):
            
            # Set up a plotter for rendering
            plotter = pv.Plotter(off_screen=True, window_size=(640, 640))
            plotter.background_color = 'white'  # Set a white background color

            # Render the scene
            #plotter.add_mesh(mesh, show_edges=False, color='gray', specular=1.0)  # Set color and specular properties

            # Check if the mesh has color data
            if 'colors' in mesh.point_data:
                # Use the existing colors
                plotter.add_mesh(mesh, show_edges=False, specular=1)
            else:
                # Apply a default color if no colors data
                plotter.add_mesh(mesh, color='gray', show_edges=False, specular=1)

            # Set the viewing angle
            plotter.camera_position = 'xy'
            plotter.camera.azimuth += azim_angle
            plotter.camera.elevation += elev_angle
            plotter.camera.roll += roll_angle
            #print(angle)

            #DISABLED OUTPUT PREVIEW
            #plotter.show(auto_close=False)

            # Save the rendered image
            filename = f'SHD_rot_a{azim_angle}_e{elev_angle}_r{roll_angle}.png'
            output_file = os.path.join(OUTPUT_DIR, filename)
            plotter.screenshot(output_file)
                
            # Close the plotter
            plotter.close()

### 3. Creation of Labels
After creating the collection of images, the final step to complete the dataset is generating a collection of labels. Labeling is typical in object detection; each label is a .txt file containing the class of the object and the coordinates of the area where it is located. Accurate labels ensure that the model learns to correctly identify objects, their locations, and their boundaries, directly impacting the model's performance and reliability in real-world applications. Mislabeling or inconsistent labeling can lead to poor model accuracy, making it less effective at detecting and categorizing objects correctly.

In this particular case, I am following the standard format of YOLOv8, with the following class assignment:

- QD = 0
- QF = 1
- QFA = 2
- SD = 3
- SHD = 4

In [31]:
!pip install opencv-python

     --------------------------------------- 38.8/38.8 MB 13.1 MB/s eta 0:00:00


In [32]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [33]:
def create_labels(image_path, label_path, current_category=0):
    if not os.path.isfile(image_path):
        print(f"File not found: {image_path}")
        return
    
    frame = cv2.imread(image_path)
    img_h, img_w, _ = frame.shape

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding
    thresholded = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                        cv2.THRESH_BINARY_INV, 11, 2)

    # Apply morphological operations to close small holes in the foreground
    kernel = np.ones((5,5), np.uint8)
    morphed = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        print(f"No contours found in image: {image_path}")
        return

    # Find the largest contour which should be the object
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    x_centre = x + (w / 2)
    y_centre = y + (h / 2)
    
    # Normalization
    X = x_centre / img_w
    Y = y_centre / img_h
    W = w / img_w
    H = h / img_h

    # Limiting up to fixed number of decimal places
    X = format(X, '.6f')
    Y = format(Y, '.6f')
    W = format(W, '.6f')
    H = format(H, '.6f')
    
    with open(label_path, "w") as file_object:
        file_object.write(f"{current_category} {X} {Y} {W} {H}\n")

In [34]:
# Dictionary with magnet types as keys and classes as values
magnet_classes = {
    'QD': 0,
    'QF': 1,
    'QFA': 2,
    'SD': 3,
    'SHD': 4,
}

In [35]:
# Define work folders
input_path = 'data/CAD_pics'
output_path = 'data/labels'

In [36]:
os.makedirs(output_path, exist_ok=True)

# Process each image in the directory
for filename in tqdm(os.listdir(input_path)):
    if filename.endswith('.png'):
        magnet_type = filename.split('_rot')[0]
        magnet_class = magnet_classes.get(magnet_type, None)
        if magnet_class is not None:
            img_path = os.path.join(input_path, filename)
            label_path = os.path.join(output_path, filename.replace('.png', '.txt'))
            create_labels(img_path, label_path, magnet_class)
        else:
            print(f'Warning: Magnet type {magnet_type} not found in dictionary.')

print('Label generation complete.')

100%|██████████| 4800/4800 [02:09<00:00, 37.10it/s]

Label generation complete.
